In [32]:

import numpy as np
import pandas as pd
import scipy
import librosa
import librosa.display
import random
import matplotlib.pyplot as plt
import sklearn

import os
from tqdm import tqdm
from matplotlib import cm, colors, pyplot as plt
from matplotlib.colors import ListedColormap

import IPython 
from IPython.display import display


import tensorflow as tf
import tensorflow.python.keras as k

from tensorflow import keras
from tensorflow.keras import layers

from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


# https://www.kaggle.com/annashtangey/music-genre-classification-coeff-experiment-94-4

In [33]:
sr = 22050 
sample_rate = sr    # sample rate. based on Nyquist frequency, we only care about frequencies up to 10kHz therefor the sample rate will only perserve those frequencies 
n_fft = 2048
hop_length = 512
duration = 29 # length of song to be used (in seconds) 
n_mels=128
seed = 42
random.seed = seed
np.random.seed(seed)

In [34]:
# load metadata csv
df = pd.read_csv('/Users/stephen/Flatiron/Phase4/MIR/Data/features_30_sec.csv')
df = df[['filename', 'label']]

# relative path to the directory of audio file genre subfolders
rel_path = '/Users/stephen/Flatiron/Phase4/MIR/Data/genres_original'

# create relative path to each audio file
df['file_path'] = rel_path + df['label'] + '/' + df['filename']

# drop 'filename' column
#df = df.drop(columns='filename')
df.head()

,filename,label,file_path
0,blues.00000.wav,blues,/Users/stephen/Flatiron/Phase4/MIR/Data/genres...
1,blues.00001.wav,blues,/Users/stephen/Flatiron/Phase4/MIR/Data/genres...
2,blues.00002.wav,blues,/Users/stephen/Flatiron/Phase4/MIR/Data/genres...
3,blues.00003.wav,blues,/Users/stephen/Flatiron/Phase4/MIR/Data/genres...
4,blues.00004.wav,blues,/Users/stephen/Flatiron/Phase4/MIR/Data/genres...


In [35]:
# drop the corrupted song
df = df.drop(labels=554)

# check
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   filename   999 non-null    object
 1   label      999 non-null    object
 2   file_path  999 non-null    object
dtypes: object(3)
memory usage: 31.2+ KB


In [36]:
def get_mfcc(filename, sample_rate=sr):
    y, sr = librosa.load(
        path = filename,  # load in audio file. MP3 not supported refer to Librosa documentation 
        sr = sample_rate, # by convention the default sample rate is 22050, lower if not enough processing power 
        mono = True,      # stereo isn't important. 
        offset = 0.0,     # start reading audio after this time (in seconds)
        duration = duration, 
        res_type = 'kaiser_best'
        )

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mels=n_mels)
    return mfcc

In [44]:
audio_dataset_path = '/Users/stephen/Flatiron/Phase4/MIR/Data/genres_original/'


mfccs=[]
for index_num,row in tqdm(df.iterrows()):
    try:
        final_class_labels=row["label"]
        another_label=row['file_path']
        file_name = os.path.join(os.path.abspath(audio_dataset_path), final_class_labels+'/',str(row["filename"]))    
        data=get_mfcc(file_name)
        mfccs.append([data,final_class_labels,another_label])
        
    except Exception as e:
        print(f"Error: {e}")
        continue

999it [00:35, 27.96it/s]
